In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
import math
def calculate_angle(a, b):
    a = np.array(a)
    b = np.array(b)
    vertical = np.array([0, 1])
    
    ba = a - b

    mag_ba = math.sqrt(ba[0]**2 + ba[1]**2)
    mag_vertical = 1
    
    radians = math.acos(np.dot(ba, vertical) / (mag_ba * mag_vertical))
    
    angle = np.abs(radians * 180 / np.pi)
    
    if angle > 180:
        angle = 360 - angle
    
    return angle

In [8]:
import math

def calculate_angle_alt(shoulder, hip):
    x1, y1 = shoulder
    x2, y2 = hip

    vector = (x1 - x2, y1 - y2)

    vertical_vector = (0, -1)

    dot_product = vector[0] * vertical_vector[0] + vector[1] * vertical_vector[1]

    vector_magnitude = math.sqrt(vector[0]**2 + vector[1]**2)
    vertical_magnitude = 1

    angle_rad = math.acos(dot_product / (vector_magnitude * vertical_magnitude))

    angle_deg = math.degrees(angle_rad)

    return angle_deg

In [5]:
calculate_angle_alt((2,4),(4,2))

In [6]:
def draw_landmarks(image, landmarks, selected_landmarks, connections):
    for idx in selected_landmarks:
        lm = landmarks[idx]
        h, w, _ = image.shape
        cx, cy = int(lm.x * w), int(lm.y * h)
        cv2.circle(image, (cx, cy), 5, (0, 255, 0), -1)

    for connection in connections:
        start_idx, end_idx = connection
        if start_idx in selected_landmarks and end_idx in selected_landmarks:
            start_lm = landmarks[start_idx]
            end_lm = landmarks[end_idx]

            start_coords = (int(start_lm.x * w), int(start_lm.y * h))
            end_coords = (int(end_lm.x * w), int(end_lm.y * h))
            cv2.line(image, start_coords, end_coords, (255, 0, 0), 2)
            
selected_landmarks = [
    mp_pose.PoseLandmark.RIGHT_SHOULDER.value,
    mp_pose.PoseLandmark.RIGHT_HIP.value,
    mp_pose.PoseLandmark.RIGHT_KNEE.value,
    mp_pose.PoseLandmark.RIGHT_ANKLE.value,
    mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value,
    # mp_pose.PoseLandmark.LEFT_SHOULDER.value,
    # mp_pose.PoseLandmark.LEFT_HIP.value,
    # mp_pose.PoseLandmark.LEFT_KNEE.value,
    # mp_pose.PoseLandmark.LEFT_ANKLE.value,
    # mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value,
]

connections = [
    (mp_pose.PoseLandmark.RIGHT_SHOULDER.value, mp_pose.PoseLandmark.RIGHT_HIP.value),
    (mp_pose.PoseLandmark.RIGHT_HIP.value, mp_pose.PoseLandmark.RIGHT_KNEE.value),
    (mp_pose.PoseLandmark.RIGHT_KNEE.value, mp_pose.PoseLandmark.RIGHT_ANKLE.value),
    (mp_pose.PoseLandmark.RIGHT_ANKLE.value, mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value),
    # (mp_pose.PoseLandmark.LEFT_SHOULDER.value, mp_pose.PoseLandmark.LEFT_HIP.value),
    # (mp_pose.PoseLandmark.LEFT_HIP.value, mp_pose.PoseLandmark.LEFT_KNEE.value),
    # (mp_pose.PoseLandmark.LEFT_KNEE.value, mp_pose.PoseLandmark.LEFT_ANKLE.value),
    # (mp_pose.PoseLandmark.LEFT_ANKLE.value, mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value)
]

In [9]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    counter = 0
    stage = None
    up = True
    down = False
    while cap.isOpened():
        ret, frame = cap.read()
    
        frame = cv2.flip(frame, 1)
    
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
    
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
        try:
            landmarks = results.pose_landmarks.landmark
    
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            shoulder_hip_vertical = calculate_angle_alt(right_shoulder, right_hip)
            hip_knee_vertical = calculate_angle_alt(right_hip, right_knee)
            knee_ankle_vertical = calculate_angle_alt(right_knee, right_ankle)
            
            cv2.putText(image, f'SH-HIP: {shoulder_hip_vertical:.1f}', 
                        tuple(np.multiply(right_hip, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f'HIP-KNEE: {hip_knee_vertical:.1f}',
                        tuple(np.multiply(right_knee, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f'KNEE-ANKLE: {knee_ankle_vertical:.1f}',
                        tuple(np.multiply(right_ankle, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            feedback = []
            if shoulder_hip_vertical < 10:
                feedback.append("Bend forward")
            if shoulder_hip_vertical > 25:
                feedback.append("Bend backward")
            if 50 < hip_knee_vertical < 80:
                feedback.append("Lower your hips")
            if hip_knee_vertical > 95:
                feedback.append("Knees falling over toes")
            if knee_ankle_vertical > 30:
                feedback.append("Squats too deep")
            
            for i, msg in enumerate(feedback):
                cv2.putText(image, msg, (10, 100 + i * 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)

            if up:
                if hip_knee_vertical < 32:
                    stage = "stage1"  # up
                elif stage == "stage1" and 35 < hip_knee_vertical < 65:
                    stage = "stage2"  # middle
                elif stage == "stage2" and 75 < hip_knee_vertical < 95:
                    stage = "stage3"  # down  
                    down = True
                    up = False
                    
            if down:
                if stage == "stage3" and 35 < hip_knee_vertical < 65:
                    stage = "stage2"
                elif stage == "stage2" and hip_knee_vertical < 32:
                    stage = "stage1"
                    down = False
                    up = True
                    counter += 1     

            cv2.putText(image, 'Count: ', (15, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
                
        except:
            pass

        draw_landmarks(image, landmarks, selected_landmarks, connections)
    
        cv2.imshow('Mediapipe Feed', image)
    
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
